In [1]:
import pickle
import numpy as np
from gensim.models import Doc2Vec as d2v
from gensim.models.doc2vec import TaggedDocument as td
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

from pprint import pprint
from time import time
import logging

In [2]:
# #load all of our documents
# with open('NHlegtext09_20.corpus','rb') as f:
#     legcorpus = pickle.load(f)
    
# # #print a random doc
# # print(legcorpus[303])

# #assemble the text data, and remove the part where it says if it passes or not
# combined_docs = {} #{bill_id:text}
# total = 0
# passed = 0

# #a more unorthodox method of grabbing the target passage
# leg_target = {} #{bill: 0 or 1, where 1 means pass}

# #for each document, check if it is approved, record that, then combine the list into a string
# for doc in legcorpus:
#     pass_flag = 0
#     if doc[0] not in combined_docs:
#         total += 1
#         try:
#             if doc[2][-5] == 'approved':
#                 doc[2] = doc[2][:-5]
#                 passed += 1
#                 pass_flag = 1
#                 combined_docs[doc[0]] = " ".join(doc[2])
#             else:
#                 combined_docs[doc[0]] = " ".join(doc[2])
                
#         except:
#             combined_docs[doc[0]] = " ".join(doc[2])
#             pass
                
#     leg_target[doc[0]] = pass_flag
#     #for now, ignoring the amendments, but it seems like it could do better with the amendments
#     #that would go here (i.e. an else statement saying doc[0] in combined_docs)
    
                   
# print("\nTotal bills:", total)
# print("Passed bills:", passed)
# print("Bill passed ratio:", passed/total)

In [5]:
#actually, lets just stick with the given passages for now
leg_data = pd.read_csv("FedLegOverallData09_20.csv") 

#with open('NHStateTargetList09_20.list','rb') as f:
#    event_target = pickle.load(f)

with open('FedWinList09_20.list','rb') as f:
    win_state = pickle.load(f)
    
    
leg_targ = pd.DataFrame({"every stage passed":win_state}) 
#leg_targ = #append bill_id

bill_order = []
bill_target = []
bill_texts = []

#leg_targ_old = leg_target
leg_target = {}

####Fix this when we can load in bill texts
# for i, bill_id in enumerate(leg_data["bill_id"]):
#     if str(bill_id) not in bill_order:
#         bill_order.append(str(bill_id))
#         bill_target.append(leg_targ["every stage passed"][i])
#         leg_target[str(bill_id)] = leg_targ["every stage passed"][i]
        
#         if bill_id in combined_docs:
#             bill_texts.append(combined_docs[bill_id])
#         else:
#             bill_texts.append("")

# #for i, bill in enumerate()
# print(len(bill_texts))    
# print(len(bill_target))
# print(len(leg_data))
# # 
# for bill_id in bill_order:
#     if bill_id in combined_docs:
#         bill_texts.append(combined_docs[bill_id])
#     else:
#         bill_texts.append("")
# passed = 0
# total = 0
# for i in leg_target:
#     passed+=leg_target[i]
#     total += 1
# print(passed/total)
#print(leg_target)

In [4]:
#print(leg_targ[2100:2120])

In [6]:
#features are in a dictionary. 
#with open('BillFeatures.dict','rb') as f:
#    features = pickle.load(f)
features = leg_data
print(len(leg_targ))
X = pd.DataFrame(features)
X['leg_targ'] = leg_targ
# X_all = X.copy()
# X_all = X_all.set_index('bill_id')


#X = X.drop_duplicates(subset=['title']) #should probably be bill_id
X = X.set_index('bill_id')
# X = X.drop(columns=['time_t', 'event'])


# print(len(X))
# #print(X['committee introduced'].head())


# bill_order = []
# bill_target = []
# bill_texts = []

# count = 0
# for bill_id in X.index:
#     if bill_id in combined_docs:
#         bill_texts.append(combined_docs[bill_id])
#     else:
#         bill_texts.append("")
        
# print(len(bill_texts))
# X['texts'] = bill_texts
# # leg_targ_old = leg_target
# # leg_target = {}
# # for i, bill_id in enumerate(leg_data["bill_id"]):
# #     if str(bill_id) not in bill_order:
# #         bill_order.append(str(bill_id))
# #         bill_target.append(leg_targ["every stage passed"][i])
# #         leg_target[str(bill_id)] = leg_targ["every stage passed"][i]
        
# #         if bill_id in combined_docs:
# #             bill_texts.append(combined_docs[bill_id])
# #         else:
# #             bill_texts.append("")

# X.head()

69676


In [12]:
# #make a mask so we only keep the rows that we actually have text for
# X_mask = [False]*len(X)
# for i, bill_id in enumerate(combined_docs): 
#     try:
#         print(X['bill_id'][X['bill_id'] == bill_id].index)
#         X['bill_id'][X['bill_id'] == bill_id].index[0]
#         X_mask[X['bill_id'][X['bill_id'] == bill_id].index[0]] = True
#     except:
#         print("i:",i,"bill_id:",bill_id)
#         raise IndexError 

# #next step: sift X so that it only contains ones we have docs for
# X_mask = pd.Series(X_mask)
# print(X_mask.iloc[4:9])
# X = X[X_mask]

#X = X.astype({'bill_id': 'int64'}).sort_values('bill_id', ascending=True, ignore_index=True)
#temporarily affix the target
#X['target'] = leg_targ
#X['text'] = bill_texts
X = X.astype({'year': 'int64'}).sort_values('year', ascending=True)#, ignore_index=True)
X = X.astype({'numeric_date': 'float'}).sort_values('numeric_date', ascending=True)
#date = #make date introduced something like 2018.50, so that we can sort numerically by month too


#print(X.loc[894687]['title'])
X.tail()

Unnamed: 0  numeric_date  sponsor_vec  session  year bill_type  \
bill_id                                                                   
1293193       69673   2019.975806     1.000000     1658  2019         B   
1271391       66257   2019.975806     0.500000     1658  2019         B   
1292586       69577   2019.975806     0.500000     1658  2019         B   
1294785       69674   2020.005376     0.071429     1658  2020         B   
1294855       69675   2020.005376     1.000000     1658  2020         B   

                                                     title proposed_chamber  \
bill_id                                                                       
1293193                       Conan Commemorative Coin Act                H   
1271391  A bill to amend title 38, United States Code, ...                S   
1292586  Improving Safety and Security for Veterans Act...                S   
1294785  To require the Administrator of the Environmen...                H   
1294855  To amend the Congressional Budget Act of 1974 ...                H   

               primary_name primary_id primary_party  number_sponsors  \
bill_id                                                                 
1293193       Steve Watkins      20122             R                1   
1271391        John Boozman       9210             R                2   
1292586         Joe Manchin       9625             D                2   
1294785  Christopher Pappas      20091             D               14   
1294855       Bradley Byrne      16101             R                1   

                                      committee_introduced month_introduced  \
bill_id                                                                       
1293193              House Committee on Financial Services               12   
1271391                                               None               12   
1292586                                               None               12   
1294785  House Committee on Transportation and Infrastr...                1   
1294855                                 Committee on Rules                1   

         leg_targ  
bill_id            
1293193         0  
1271391         0  
1292586         0  
1294785         0  
1294855         0

In [13]:
len(X)

69676

In [14]:
#Change this so it isn't so memory intensive in the future: combine the text later

# #align the documents with their row to make one complete dataset
# ordered_docs = [""]*len(X)

# #align the targets with their row
# ordered_target = [0]*len(X)
# #print(leg_target)

# #take the bill ids from the dataframe (which are in order)
# ordered_bill_ids = list(X.index.values)
# for i, b_id in enumerate(ordered_bill_ids):
#     try:
#         ordered_docs[i] = combined_docs[b_id]
#     except:
#         ordered_docs[i] = ""
        
#     ordered_target[i] = leg_target[str(b_id)]
#     if str(b_id) == '894761':
#         print(i)
#         print(ordered_target[i])
    

# #make a new column for text    
# X['legtext'] = ordered_docs

In [15]:
#list(X.index.values)[0:10]
#ordered_target[0]
X['month_introduced'] = X['month_introduced'].astype(str)

X.tail()

Unnamed: 0  numeric_date  sponsor_vec  session  year bill_type  \
bill_id                                                                   
1293193       69673   2019.975806     1.000000     1658  2019         B   
1271391       66257   2019.975806     0.500000     1658  2019         B   
1292586       69577   2019.975806     0.500000     1658  2019         B   
1294785       69674   2020.005376     0.071429     1658  2020         B   
1294855       69675   2020.005376     1.000000     1658  2020         B   

                                                     title proposed_chamber  \
bill_id                                                                       
1293193                       Conan Commemorative Coin Act                H   
1271391  A bill to amend title 38, United States Code, ...                S   
1292586  Improving Safety and Security for Veterans Act...                S   
1294785  To require the Administrator of the Environmen...                H   
1294855  To amend the Congressional Budget Act of 1974 ...                H   

               primary_name primary_id primary_party  number_sponsors  \
bill_id                                                                 
1293193       Steve Watkins      20122             R                1   
1271391        John Boozman       9210             R                2   
1292586         Joe Manchin       9625             D                2   
1294785  Christopher Pappas      20091             D               14   
1294855       Bradley Byrne      16101             R                1   

                                      committee_introduced month_introduced  \
bill_id                                                                       
1293193              House Committee on Financial Services               12   
1271391                                               None               12   
1292586                                               None               12   
1294785  House Committee on Transportation and Infrastr...                1   
1294855                                 Committee on Rules                1   

         leg_targ  
bill_id            
1293193         0  
1271391         0  
1292586         0  
1294785         0  
1294855         0

In [10]:
# list(combined_docs.keys())[0:10]
# combined_docs[576685]
# #ordered_target
# X['legtext'].head()

In [11]:
#create the entries for the data for demo
# new_test_data = {'bill_id':[],
#                  'sponsor_vec':[],
#                  'chamber':[],
#                  'bill_type':[],
#                  'proposed_chamber':[],
#                  'primary_name':[],
#                  'primary_party':[],
#                  'number_sponsors':[],
#                  'committee_introduced':[],
#                  'month_introduced':[],}            

#HB1680: bill_id is 1294143
#HB1389: bill_id is 1287298

In [62]:
#select the data that we actually want to train on
#it is probably better to use primary id in the future instead of primary name
#X_new = X[['house/sentate', 'primary name', 'primary party', 'number of co sponsors', 'committee introduced','month introduced']].copy() #, 'committee introduced'
X_new = X[['sponsor_vec', 'bill_type', 'proposed_chamber', 'primary_name','primary_party','number_sponsors','committee_introduced','month_introduced']].copy()                      
#print(ordered_target[0:5])

train_start = 50000
amount_train = 14000
amount_test = 1000

X_dum = pd.get_dummies(X_new)

#X_dum = pd.concat([X_dum_2, pd.get_dummies(X['event'])], axis=1)

#print(X_dum.iloc[0])

#ordered_target = pd.get_dummies(X['target'])

X_train_p = X_dum.iloc[train_start:train_start+amount_train]#.iloc
y_train = X['leg_targ'].iloc[train_start:train_start+amount_train]#pd.get_dummies()
X_test_p = X_dum.iloc[train_start+amount_train:train_start+amount_train+amount_test]#.iloc
y_test = X['leg_targ'].iloc[train_start+amount_train:train_start+amount_train+amount_test]#pd.get_dummies()


k_select = SelectKBest(f_classif, k=500)
X_train = k_select.fit_transform(X_train_p, y_train)
X_test = k_select.transform(X_test_p)

indices = k_select.get_support(indices=True)
print(X_train_p.iloc[0,indices])
# #indices = 

print(sum(y_train)/len(y_train))
print(sum(y_test)/len(y_test))
#print(X_train[0:10])
#print(X_test[0:10])
#X_train.head()


sponsor_vec            0.333333
number_sponsors       27.000000
bill_type_B            1.000000
bill_type_CR           0.000000
bill_type_JR           0.000000
                        ...    
month_introduced_5     1.000000
month_introduced_6     0.000000
month_introduced_7     0.000000
month_introduced_8     0.000000
month_introduced_9     0.000000
Name: 1015300, Length: 500, dtype: float64
0.039142857142857146
0.019


/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [   8   12   20   22   23   28   29   30   35   43   44   52   54   55
   58   61   62   63   65   66   67   73   74   78   80   82   94   96
   97  100  101  104  105  106  116  117  122  126  128  129  130  135
  136  140  141  142  144  146  147  149  151  154  155  161  162  164
  165  166  173  177  181  183  186  187  188  189  190  191  192  193
  195  199  200  202  205  206  213  215  216  221  225  233  234  244
  246  247  248  251  257  260  264  268  269  282  284  286  288  289
  299  301  302  304  305  307  310  313  315  316  317  318  322  323
  328  331  332  336  337  339  340  341  342  345  347  349  353  361
  364  368  371  372  373  374  376  379  380  381  382  383  384  385
  386  390  397  400  406  407  412  413  418  419  423  424  427  433
  434  436  440  441  444  447  448  451  452  454  455  456  457  460
  461  465  470  472  473  475 

In [48]:
for i in indices:
    print(X_train_p.columns[i], "\t", X_train_p.iloc[0,i])

sponsor_vec 	 0.9090909090909092
number_sponsors 	 11
bill_type_B 	 1
bill_type_JR 	 0
bill_type_R 	 0
proposed_chamber_H 	 1
proposed_chamber_S 	 0
primary_name_Angela Craig 	 0
primary_name_Ann Wagner 	 0
primary_name_Barbara Comstock 	 0
primary_name_Bill Huizenga 	 0
primary_name_Bill Shuster 	 0
primary_name_Blaine Luetkemeyer 	 0
primary_name_Bradley Byrne 	 0
primary_name_Brett Guthrie 	 0
primary_name_Brian Fitzpatrick 	 0
primary_name_Bruce Poliquin 	 0
primary_name_Chuck Grassley 	 0
primary_name_Colin Allred 	 0
primary_name_Dan Newhouse 	 0
primary_name_David Roe 	 0
primary_name_Devin Nunes 	 0
primary_name_Doug Collins 	 0
primary_name_Edward Royce 	 0
primary_name_Eleanor Norton 	 0
primary_name_Elizabeth Warren 	 0
primary_name_Frank LoBiondo 	 0
primary_name_French Hill 	 0
primary_name_Garland Barr 	 0
primary_name_Greg Walden 	 0
primary_name_Jaime Herrera Beutler 	 0
primary_name_James McGovern 	 0
primary_name_Jamie Raskin 	 0
primary_name_Jason Chaffetz 	 1
primar

In [57]:
#X_new.loc[243547]

In [58]:
#compressed_legtext = pd.DataFrame(X['texts'].copy())


In [59]:
#prepare the documents by using tfidf vectors:

# #from https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a
# stemmer = SnowballStemmer("english", ignore_stopwords=True)
# #adjust this stemmer so that it does more than one word in the tokens

# class StemmedCountVectorizer(CountVectorizer):
#     def build_analyzer(self):
#         analyzer = super(StemmedCountVectorizer, self).build_analyzer()
#         return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

# #mini_train = int(amount_train/len(X))*len(compressed_legtext)
# #mini_test = int(amount_test/len(X))*len(compressed_legtext)

# stemmed_count_vect = StemmedCountVectorizer(stop_words='english', ngram_range=(1,2))
# stem_counts = stemmed_count_vect.fit_transform(compressed_legtext['texts'][0:amount_train])
# stem_counts_test = stemmed_count_vect.transform(compressed_legtext['texts'][amount_train:amount_train+amount_test])

# tfidf = TfidfTransformer()
# tfidf_vectors = tfidf.fit_transform(stem_counts)
# tfidf_vectors_test = tfidf.transform(stem_counts_test)

In [60]:
# #do the final pick after, since we only want to run the above cell once
# k_best_tfidf = SelectKBest(f_classif, k=1000)
# final_tfidf_features = k_best_tfidf.fit_transform(tfidf_vectors, y_train)
# final_tfidf_features_test = k_best_tfidf.transform(tfidf_vectors_test)
# word_indices = k_best_tfidf.get_support(indices=True)
# #print(word_indices)

In [19]:
#### print(stemmed_count_vect.__dict__.keys())
#print(stemmed_count_vect.__dict__['vocabulary_']['rsa'])


445962


In [20]:
#try withouth the words
#try with the state to state
#train the earlier fit with the overall classifier, it doesnt work with state to state

In [61]:
#combine tfidf legtext and the other features:
# legtext_train = final_tfidf_features.todense()
# #legtext_train = pd.DataFrame(legtext_train)#*10

# legtext_test = final_tfidf_features_test.todense()
# #legtext_test = pd.DataFrame(legtext_test)


# X_train = np.concatenate([np.array(X_train),legtext_train], axis=1)
# X_test = np.concatenate([np.array(X_test),legtext_test], axis=1)
#X_train.head()
#X_train[0]

In [20]:
# def perf_metric(y_test, y_pred):
# #Accuracy
#     correct=0
#     total=0
#     acc=0
#     for i in range(len(y_test)):
#         for j in range(len(y_test.iloc[0])):
#             answer = y_test.iloc[i,j] 
#             real = y_pred[i][j]:
#             if answer == real:
#                 correct+=real 
#         total+=1
#     acc=correct/total*100
#     #print(x,y,len(y_pred),len(y_test))
#     #print('Accuracy:', acc)
#     return acc

def perf_metric(y_test, y_pred):
    y_test = np.array(y_test)
    y_pred = np.array(y_pred)
    y_arg_test = np.argwhere(y_test == 1)[:,1]
    print("y_arg_test[0:10] :",y_arg_test[0:10])
    print("len(y_arg_test): ",len(y_arg_test))
    y_arg_pred = np.argwhere(y_pred == 1)
    _pred = [-1]*len(y_arg_test)
    print("Doing got loop...")
    for i, prediction in enumerate(y_arg_pred):
         _pred[prediction[0]] = prediction[1]
    print("Done with for loop")
    y_arg_pred = _pred
    
    print("y_arg_pred[0:10] :",y_arg_pred[0:10])
    print("len(y_arg_pred): ",len(y_arg_pred))
    y_correct = y_arg_test == y_arg_pred
    acc = np.mean(y_correct)
    return acc

In [23]:
#print(np.array(y_test.iloc[-8]))
#print(predicted[1001])

In [68]:
#train the neural network a bunch
means = []
for i in range(1):
    #print('Training...')
    clf = MLPClassifier(random_state=101+i, max_iter=10)#MultinomialNB(alpha=0)#####alpha=0.001,DecisionTreeClassifier()#MultinomialNB()##19 ##
    clf.fit(X_train, y_train)
    #print('Predicting...')
    predicted = clf.predict(X_test)
    predicted_prob = clf.predict_proba(X_test)
    #print(predicted_prob[0])
    #train_prediction = clf.predict(X_train)
    #print('Mean Calculation...')
    #new_mean = perf_metric(y_test, predicted)
    new_mean = np.mean(predicted == y_test)
    means.append(new_mean)
    print(new_mean)
    #print(np.mean(train_prediction == y_train))
    
mean=np.mean(np.array(means))
print("overall mean:", mean)


0.982
overall mean: 0.982


/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [22]:
print(np.mean(y_train))
np.mean(y_test)

0.03393220338983051


0.0338

In [72]:
#actualMNBclf = MNBclf
#MNBclf = clf
#NNclf = clf

print(type(NNclf))
print(type(MNBclf))
#find bill_id from mongoDB:
#SB2968: bill_id is 1286547


bill_id = 1286547
SB_test = X_dum.loc[[bill_id]]#.reshape(1, -1)
print("All data on bill_id", bill_id)
print(X.loc[[bill_id]])
SB_test = k_select.transform(SB_test)
indices = k_select.get_support(indices=True)
print("\n\nUsed data on bill_id", bill_id)
print(X_dum.loc[[bill_id]].iloc[0,indices])

SB_test = np.array(SB_test)

# HB_legtext_counts = stemmed_count_vect.transform(X[['texts']].loc[bill_id])
# HB_legtext_tfidf = tfidf.transform(HB_legtext_counts)
# HB_legtext = k_best_tfidf.transform(HB_legtext_tfidf).todense()
# HB_legtext = np.array(HB_legtext)

SB_data = SB_test #np.concatenate([HB_test, HB_legtext], axis=1)
print(SB_data)

SB_2968_prob_MNB = MNBclf.predict_proba(SB_data)
SB_2968_prob_NN = NNclf.predict_proba(SB_data)

# HB1389_prob = MNBclf.predict_proba(HB_data)
# #HB1680_prob = MNBclf.predict_proba(X_dum.loc[1294143])

print(SB_2968_prob_MNB)
print(SB_2968_prob_NN)
# #show all the data
# print(X.loc[bill_id])

<class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>
<class 'sklearn.naive_bayes.MultinomialNB'>
All data on bill_id 1286547
         Unnamed: 0  numeric_date  sponsor_vec  session  year bill_type  \
bill_id                                                                   
1286547       69170   2019.922043          0.0     1658  2019         B   

                                      title proposed_chamber    primary_name  \
bill_id                                                                        
1286547  Consumer Online Privacy Rights Act                S  Maria Cantwell   

        primary_id primary_party  number_sponsors committee_introduced  \
bill_id                                                                  
1286547       9427             D                4                 None   

        month_introduced  leg_targ  
bill_id                             
1286547               12         0  


Used data on bill_id 1286547
sponsor_vec           0.0


In [27]:
#save the state-to-state classifier
#StSclf = clf #this has been run and saved

In [28]:
#y_test
def vec_to_state(prediction_vectors, example_output_df):
    index = np.array(example_output_df.columns)
    print('index[0:10]',index[0:10])
    #print(index)
    #prediction_vectors = np.array([[1,0,0,0,0], [0,0,1,0,0], [0,0,0,0,1],[0,1,0,0,0]]) == 1
    #print(prediction_vectors)
    prediction_vectors = prediction_vectors == 1
    predictions = []
    for prediction in prediction_vectors:
        #print(prediction)
        
        if len(index[prediction]) == 0:
            append_value = [0]
        else:
            append_value = index[prediction][0]
            
        print('append_value:', append_value)
        predictions.append(append_value)
        
    return predictions

def prob_vec_to_n_states(prediction_vector, example_output_df, n=1):
    #return the names, and probabilities in order
    index = np.array(example_output_df.columns)
    
    #print(index)
    #prediction_vectors = np.array([[1,0,0,0,0], [0,0,1,0,0], [0,0,0,0,1],[0,1,0,0,0]]) == 1
    #print(prediction_vectors)
    ind = np.argpartition(prediction_vector, -n)[-n:]
    sorted_ind = ind[np.argsort(prediction_vector[ind])][::-1]
    
    predictions = []
    for index_s in sorted_ind:
        append_value = index[index_s]
            
        print('append_value:', append_value)
        predictions.append((append_value, prediction_vector[index_s]))
        
    return predictions
    


#pd.get_dummies(X['event'])
def state_to_vec(current_state, example_output_df):
    X_index = np.array(example_output_df.columns)
    indices = current_state #a string

    #for index in indices:
    location = X_index == index
    location = location.astype(int)
        #print(location)
    return(location)
    

In [50]:
#pd.get_dummies(X[['event', 'month_introduced']]).columns

In [ ]:
#HB1680: bill_id is 1294143
#HB1389: bill_id is 1287298

#save the state to state data so we can make a prediction on it next
#sta
#see what is actually in there
#print(X_dum.loc[1294143])
#print(X_dum.loc[1287298])
#print(X.loc[1294143])
#prediction_vector = StSclf.predict_proba(X_dum.loc[1294143])[-1]#1294143])
#make an initial prediction of the next step
#print(prob_vec_to_n_states(prediction_vector, y_test, n=5))

#predictions = vec_to_state(prediction_vectors, y_test)
#print(predictions)
#X_dum.head()

In [ ]:
#DTClf has been saved
#DTClf = clf

In [73]:

num_k = 500
num_words = 0

clf = MNBclf
#only run this if using MultinomialNB classifier
feature_indices = indices
pass_probs = 10**clf.feature_log_prob_[1] - 10**clf.feature_log_prob_[0]

metadata_feature_pass_probs = {X_dum.columns[feature]:pass_probs[i] for i, feature in enumerate(feature_indices)}
#inverted_vocab = stemmed_count_vect.__dict__['vocabulary_']
#vocab = {inverted_vocab[key]:key for key in inverted_vocab.keys()}

#text_feature_pass_probs = {vocab[feature]:pass_probs[i+num_k] for i, feature in enumerate(word_indices)}
#stemmed_count_vect.__dict__['vocabulary_']['rsa']

all_features = {**metadata_feature_pass_probs}#, **text_feature_pass_probs}

list_features = list(all_features.keys())

sorted_features = sorted(list_features, key=lambda x: all_features[x], reverse=True)
bad_features = sorted(list_features, key=lambda x: all_features[x], reverse=False)
#print(sorted_features)

#vocab_dict = {indices[i]:X_dum.keys()[indices][i] for i, _ in enumerate(indices)}
#print(vocab_dict)

print("Increase the amount of these features:")
for i, feature in enumerate(sorted_features):
    if i >= 50:
        break
    print("{0:.10f}".format(all_features[feature]), "\t", feature)

print("\n\nDecrease the amount of these features:")
for i, feature in enumerate(bad_features):
    if i >= 50:
        break
    print("{0:.10f}".format(all_features[feature]), "\t", feature)

Increase the amount of these features:
0.0799746169 	 number_sponsors
0.0001387900 	 primary_party_R
0.0000963906 	 proposed_chamber_H
0.0000700745 	 sponsor_vec
0.0000254243 	 bill_type_R
0.0000084447 	 month_introduced_12
0.0000007364 	 month_introduced_6
0.0000006353 	 committee_introduced_House Committee on Natural Resources
0.0000006136 	 committee_introduced_House Committee on Financial Services
0.0000002899 	 committee_introduced_House Committee on Foreign Affairs
0.0000002309 	 committee_introduced_House Committee on Homeland Security
0.0000001835 	 primary_name_Pete Sessions
0.0000001834 	 primary_name_Ken Buck
0.0000001830 	 bill_type_JR
0.0000001314 	 primary_name_Doug Collins
0.0000001309 	 primary_name_Michael Burgess
0.0000001098 	 primary_name_Rob Woodall
0.0000001093 	 primary_name_Liz Cheney
0.0000000898 	 primary_name_Dan Newhouse
0.0000000815 	 committee_introduced_Committee on Foreign Affairs
0.0000000566 	 primary_name_Tom Cole
0.0000000270 	 committee_introduced_H

In [ ]:
print(len(predicted))
print()
for i, prediction in enumerate(predicted):
    if i == 50:
        break
    print("real:", y_test[i],"\tpredicted:", predicted_prob[i][1])#, "\tdata:", X_dum.iloc[amount_train:amount_train+amount_test,indices])
